In [ ]:

%pip install -q transformers datasets accelerate peft bitsandbytes sentencepiece

In [ ]:

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from peft import (
    get_peft_model,
    LoraConfig,
    prepare_model_for_kbit_training,
    TaskType,
)
from datasets import load_dataset
import os


os.environ["TOKENIZERS_PARALLELISM"] = "false"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


model_name = "Vikhrmodels/VikhrT5-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

special_tokens = {
    "additional_special_tokens": [
        "<system>",      
        "</system>",     
        "<user>",        
        "</user>",       
        "<reasoning>",   
        "</reasoning>",  
        "<answer>",      
        "</answer>",     
    ]
}


tokenizer.add_special_tokens(special_tokens)

max_memory = {i: torch.cuda.get_device_properties(i).total_memory for i in range(torch.cuda.device_count())}
print(f"Обнаружена карта памяти: {max_memory}")

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    max_memory=max_memory
)

model.resize_token_embeddings(len(tokenizer))

print(f"Модель {model_name} загружена и распределена по GPU.")
print("Карта устройств (device_map):", model.hf_device_map)
print(f"Добавлены специальные токены: {special_tokens['additional_special_tokens']}")
print(f"Новый размер словаря: {len(tokenizer)}")

In [ ]:
def build_prompt_with_special_tokens(sample):

    system_content = (
        "**Инструкция для эксперта-аналитика**\n"
        f"Тема: '{sample.get('subject', sample.get('domain', 'Общая тема'))}'.\n"
        "Ваша задача — выполнить строгий логический анализ предоставленной задачи. Следуйте этому алгоритму:\n"
        "1.  **Анализ Задачи:** Кратко определите основной вопрос и какой логический или математический принцип нужно применить.\n"
        "2.  **Пошаговая Оценка Вариантов:** Систематически рассмотрите КАЖДЫЙ вариант ответа (A, B, C, D). Для каждого варианта предоставьте четкое и лаконичное объяснение, почему он является верным или неверным в контексте задачи.\n"
        "3.  **Синтез и Вывод:** На основе пошагового анализа, сделайте окончательный вывод и выберите единственно правильный ответ.\n\n"
        "**Формат вывода**\n"
        "Ваш ответ ДОЛЖЕН СТРОГО соответствовать формату ниже, без лишних вступлений или заключений:\n"
        "**Рассуждение:**\n"
        "[Здесь ваш детальный анализ по шагам 1-3]\n"
        "**Ответ:** [Здесь ОДНА буква: A, B, C или D]"
    )
    
    user_content = sample.get('original_question', '')
    
    input_prompt = f"<system>{system_content}</system>\n<user>{user_content}</user>"
    
    return {
        "system_prompt": system_content,
        "user_prompt": user_content,
        "input_prompt": input_prompt
    }

In [ ]:
def format_target_with_special_tokens(reasoning, answer):
    cleaned_reasoning = reasoning.strip()
    if cleaned_reasoning.startswith("**Рассуждение:**"):
        cleaned_reasoning = cleaned_reasoning.replace("**Рассуждение:**", "").strip()
    
    cleaned_answer = answer.strip()
    if cleaned_answer.startswith("**Ответ:**"):
        cleaned_answer = cleaned_answer.replace("**Ответ:**", "").strip()
    
    target_text = f"<reasoning>{cleaned_reasoning}</reasoning>\n<answer>{cleaned_answer}</answer>"
    
    return target_text

In [ ]:
def test_special_tokens():
    dataset_path = "/home/jupyter/datasphere/project/proc_eval_results_merged_qwen_14b.jsonl"
    test_dataset = load_dataset("json", data_files=dataset_path, split="train")
    
    real_sample = test_dataset[0]
    
    print(f"Колонки: {test_dataset.column_names}")
    print(f"Размер датасета: {len(test_dataset)}")
    print("\nРеальный пример:")
    for key, value in real_sample.items():
        if isinstance(value, str) and len(value) > 200:
            print(f"{key}: {value[:200]}...")
        else:
            print(f"{key}: {value}")
    print("="*60)
    
    prompt_data = build_prompt_with_special_tokens(real_sample)
    target_text = format_target_with_special_tokens(
        real_sample['cleaned_reasoning'], 
        real_sample['model_extracted_answer']
    )
    
    print("Входной промпт:")
    print(prompt_data['input_prompt'])
    print("\nЦелевой текст:")
    print(target_text)
    print("\nТокенизация входного промпта:")
    input_tokens = tokenizer(prompt_data['input_prompt'])
    print(f"Количество токенов: {len(input_tokens['input_ids'])}")
    print("Первые 20 токенов:", input_tokens['input_ids'][:20])
    print("\nТокенизация целевого текста:")
    target_tokens = tokenizer(target_text)
    print(f"Количество токенов: {len(target_tokens['input_ids'])}")
    print("Первые 20 токенов:", target_tokens['input_ids'][:20])
    
    special_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in special_tokens['additional_special_tokens']]
    print(f"\nID специальных токенов: {special_token_ids}")
    
    input_token_ids = input_tokens['input_ids']
    target_token_ids = target_tokens['input_ids']
    
    for token_id in special_token_ids:
        input_count = input_token_ids.count(token_id)
        target_count = target_token_ids.count(token_id)
        token_name = tokenizer.convert_ids_to_tokens(token_id)
        print(f"{token_name} (ID: {token_id}): {input_count} в input, {target_count} в target")
    
    return prompt_data, target_text

In [ ]:
test_special_tokens()

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "k", "v", "o"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


dataset_path = "/home/jupyter/datasphere/project/proc_eval_results_merged_qwen_14b.jsonl"
full_dataset = load_dataset("json", data_files=dataset_path, split="train")

train_test_split = full_dataset.train_test_split(test_size=0.1, seed=42)
dataset_train = train_test_split["train"]
dataset_eval = train_test_split["test"]


dataset_train = dataset_train.map(build_prompt_with_special_tokens)
dataset_eval = dataset_eval.map(build_prompt_with_special_tokens)

def preprocess_function_with_special_tokens(examples):
    model_inputs = tokenizer(examples["input_prompt"], max_length=2048, truncation=True)
    
    target_texts = []
    
    for i in range(len(examples["input_prompt"])):
        reasoning = examples["cleaned_reasoning"][i]  
        answer = examples["model_extracted_answer"][i]
        target_text = format_target_with_special_tokens(reasoning, answer)
        target_texts.append(target_text)
    

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, max_length=2048, truncation=True)
        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset_train = dataset_train.map(preprocess_function_with_special_tokens, batched=True, remove_columns=dataset_train.column_names)
tokenized_dataset_eval = dataset_eval.map(preprocess_function_with_special_tokens, batched=True, remove_columns=dataset_eval.column_names)


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="VikhrT5_3b_distilled_from_reasoning",
    group_by_length=True,
    
    per_device_train_batch_size=4,  
    gradient_accumulation_steps=2,  
    
    
    learning_rate=3e-4,             
    lr_scheduler_type="cosine",     
    num_train_epochs=2,             
    #weight_decay=0.005,
    #warmup_ratio=0.05,
    
    
    save_strategy="steps",          
    do_eval=True,                       
    
    
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=100,
    eval_steps=100,
    bf16=True,
    report_to="none",
    
    eval_strategy="steps",                
    save_steps = 200,                    
    load_best_model_at_end = True,   
    save_total_limit = 4,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train, 
    eval_dataset=tokenized_dataset_eval,   
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
train_result = trainer.train()

In [ ]:
final_adapter_path = "VikhrT5-3b-large-best-lora-adapter"
print(f"\nСохраняем лучший LoRA-адаптер в: {final_adapter_path}")


trainer.save_model(final_adapter_path)
tokenizer.save_pretrained(final_adapter_path)



print(f"лучший LoRA-адаптер сохранен в директории: {final_adapter_path}")
print("После завершения сессии Kaggle вы сможете скачать эту папку из раздела 'Output'.")
print("="*50)

In [ ]:
trainer.save_model("final_best_model") 
print("Лучшая модель сохранена в папку 'final_best_model'")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-07-24 17:10:09.290387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-24 17:10:10.080899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Загрузка модели и токенизатора...
Загрузка токенизатора из '/home/jupyter/datasphere/project/final_best_model'...
Токенизатор со специальными токенами загружен.
Загрузка базовой модели 'Vikhrmodels/VikhrT5-3b' с квантизацией...
Карта памяти для GPU: {0: '85.17GB'}


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.08s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Применение LoRA-адаптера из '/home/jupyter/datasphere/project/final_best_model'...
Модель готова к инференсу на устройстве: cuda:0
Загрузка и подготовка данных из '/home/jupyter/datasphere/project/proc_eval_results_merged_qwen_14b.jsonl'...
Установлен EOS токен для остановки генерации: '</answer>' (ID: 58247)
Результаты будут сохранены в: evaluation_results_vikhr3b_distilled.jsonl


Оценка модели:   1%|▏         | 3/210 [05:02<5:47:50, 100.82s/it, ✅=0, ❌=0, ❓=96]